# Calibration tables
This notebook is for making calibration tables for electron microscopes at the TEM Gemini Centre. It is based on the `pandas` python package, and offers a flexible alternative to e.g. excel or digital micrograph. It is intended for use with the Merlin camera on the 2100F, but can also be used for other microscopes. This notebook shows how the calibration table format can/should look like, how to generate an empty table, and how to fill the table with relevant values.

## Calibrating TEM magnifications
Calibrating TEM magnifications can be done by using a calibration grid specimen (low magnifications) or a single crystal (high magnification) by using the diffractogram (FFT) of HRTEM images.
## Calibrating Camera Lengths
Calibrating camera lengths can be done by aquiring diffraction patterns from known crystals, such as gold. Using polycrystalline gold offers a good alternative as it will provide ring-patterns that are relatively straight forward to index. Calibrating the image rotation of diffraction patterns requires a single crystal specimen however, or by observing the shadow image of a known feature inside the centre disk of a defocused CBED pattern.
## Calibrating step sizes
Calibrating step sizes in STEM or in scanning probe TEM can vary
### STEM
Calibrating STEM images is relatively straight forward - simply acquire a STEM image of features of known sizes and measure them. This can also be applied to scanning probe TEM (such as SPED).
### SPED using NanoMegas SCANNER
Calibrating the step sizes in NanoMegas Scanner is a little bit tricky. One way is to first acquire an image of a known feature (e.g. a grid-specimen), and then acquire an image with a long exposure time that shows the scanned region. The scale can then be worked out by comparing the two images and knowing how many probe positions there are in the scanned region. When setting the calibration in the software, the scale must be manually tuned, and this offers the fastest and simplest option. Alternatively, one can acuire a SPED scan of a known feature and calibrate the step sizes directly. This last option works well for recalibrating your scans or setting the correct scale for your dataset.

#### IMPORTANT
The step sizes in scanning probe TEM experiments (such as SPED) will depend on the value of condenser lens 3 (indicated by the "alpha" parameter)! Therefore, a correct calibration table should include the value of alpha as well!


In [1]:
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

from math import sqrt
from pathlib import Path

## Create empty calibration table
Calibration tables are made as pandas DataFrames where all relevant (and not so relevant) information can be stored for a given calibration measurement. In all cases, the nominal variable should be given, as well as the correct (calibrated) value. Other useful information is the scale of pixels, the image rotation, and e.g. the image that was used to perform the calibration. 

In [9]:
df = pd.DataFrame(columns = [
    'Mode',
    'Alpha',
    'Nominal Mag',
    'Magnification',
    'Mag Mode',
    'Scale (nm/px)',
    'Image Rotation (deg)',
    'Nominal Camera Length (cm)',
    'Camera Length (cm)',
    'Scale (1/Å/px)',
    'Nominal Step Size X (nm)',
    'Nominal Step Size Y (nm)',
    'Step Size X (nm)',
    'Step Size Y (nm)',
    'Scan Rotation (deg)',
    'Nominal Precession Angle (deg)',
    'Precession Angle (deg)',
    'Precession Eccentricity',
    'Nominal Spotsize (nm)',
    'Spotsize (nm)'
    'Image',
    'Operator',
    'Date',
    'Specimen',
    'HT'
])

In [10]:
df

,Mode,Alpha,Nominal Mag,Magnification,Mag Mode,Scale (nm/px),Image Rotation (deg),Nominal Camera Length (cm),Camera Length (cm),Scale (1/Å/px),...,Scan Rotation (deg),Nominal Precession Angle (deg),Precession Angle (deg),Precession Eccentricity,Nominal Spotsize (nm),Spotsize (nm)Image,Operator,Date,Specimen,HT


In [12]:
#TEM magnification table
magnifications = [8E3,10E3,12E3, 15E3,20E3,25E3,30E3,40E3]
mags = [14918, 18483, 21624, 26827, 35247, 43131, 53735, 69635]
scales = [3.69, 2.98, 2.54, 2.05, 1.56, 1.28, 1.02, 0.79]
magnifications = pd.DataFrame({
    'Mag Mode': 'MAG',
    'Nominal Mag': magnifications,
    'Magnification': mags,
    'Scale (nm/px)': scales,
    'Image Rotation (deg)': None,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

#Diffraction camera length table
camera_lengths = [8, 10, 12, 15, 20, 25, 30, 40, 50, 60, 80]#, 100, 120, 150, 200]
scales = np.array([0.0136, 0.011, 0.0097, 0.008, 0.0062, 0.0050, 0.0042, 0.0032, 0.0025, 0.0021, 0.0016])
actual_camera_lengths = [16.07, 19.319, 22.586, 27.475, 35.509, 43.942, 52.493, 69.429, 86.296, 103.702, 136.743]

camera_lengths = pd.DataFrame({
    'Mode': 'TEM',
    'Nominal Camera Length (cm)': camera_lengths,
    'Camera Length (cm)': actual_camera_lengths,
    'Scale (1/Å/px)': scales,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

#Probesize tables
spot_sizes = [0.5, 0.7, 1.0, 1.2, 1.4, 1.6]
probe_sizes1 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 1,
    'Nominal Spotsize (nm)': spot_sizes,
    'Spotsize (nm)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
probe_sizes2 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 2,
    'Nominal Spotsize (nm)': spot_sizes,
    'Spotsize (nm)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
probe_sizes3 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 3,
    'Nominal Spotsize (nm)': spot_sizes,
    'Spotsize (nm)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
probe_sizes4 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 4,
    'Nominal Spotsize (nm)': spot_sizes,
    'Spotsize (nm)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
probe_sizes5 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 5,
    'Nominal Spotsize (nm)': spot_sizes,
    'Spotsize (nm)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})


#NanoMEGAS precession tables
precession_angles = [0.5, 1.0, 1.08]
precession_angles1 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 1,
    'Nominal Precession Angle (deg)': precession_angles,
    'Precession Angle (deg)': np.nan,
    'Precession Eccentricity': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

precession_angles2 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 2,
    'Nominal Precession Angle (deg)': precession_angles,
    'Precession Angle (deg)': np.nan,
    'Precession Eccentricity': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

precession_angles3 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 3,
    'Nominal Precession Angle (deg)': precession_angles,
    'Precession Angle (deg)': np.nan,
    'Precession Eccentricity': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

precession_angles4 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 4,
    'Nominal Precession Angle (deg)': precession_angles,
    'Precession Angle (deg)': np.nan,
    'Precession Eccentricity': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

precession_angles5 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 5,
    'Nominal Precession Angle (deg)': precession_angles,
    'Precession Angle (deg)': [np.nan, np.nan, 1.98],
    'Precession Eccentricity': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})

#NanoMEGAS scanner tables
steps = np.arange(0.1, 1, 0.1)
step_sizes1 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 1,
    'Nominal Step Size X (nm)': steps,
    'Nominal Step Size Y (nm)': steps,
    'Step Size X (nm)': np.nan,
    'Step Size Y (nm)': np.nan,
    'Scan Rotation (deg)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
step_sizes2 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 2,
    'Nominal Step Size X (nm)': steps,
    'Nominal Step Size Y (nm)': steps,
    'Step Size X (nm)': np.nan,
    'Step Size Y (nm)': np.nan,
    'Scan Rotation (deg)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
step_sizes3 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 3,
    'Nominal Step Size X (nm)': steps,
    'Nominal Step Size Y (nm)': steps,
    'Step Size X (nm)': np.nan,
    'Step Size Y (nm)': np.nan,
    'Scan Rotation (deg)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
step_sizes4 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 4,
    'Nominal Step Size X (nm)': steps,
    'Nominal Step Size Y (nm)': steps,
    'Step Size X (nm)': np.nan,
    'Step Size Y (nm)': np.nan,
    'Scan Rotation (deg)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
step_sizes5 = pd.DataFrame({
    'Mode': 'NBD',
    'Alpha': 5,
    'Nominal Step Size X (nm)': steps,
    'Nominal Step Size Y (nm)': steps,
    'Step Size X (nm)': steps*1.01,
    'Step Size Y (nm)': steps*0.99,
    'Scan Rotation (deg)': np.nan,
    'Image': None,
    'Date': None,
    'Operator': None,
    'Specimen': None,
    'HT': 200E3
})
calibrations = pd.DataFrame(columns = [
    'Mode',
    'Alpha',
    'Nominal Mag',
    'Magnification',
    'Scale (nm/px)',
    'Image Rotation (deg)',
    'Nominal Camera Length (cm)',
    'Camera Length (cm)',
    'Scale (1/Å/px)',
    'Nominal Step Size X (nm)',
    'Nominal Step Size Y (nm)',
    'Step Size X (nm)',
    'Step Size Y (nm)',
    'Scan Rotation (deg)',
    'Nominal Precession Angle (deg)',
    'Precession Angle (deg)',
    'Precession Eccentricity',
    'Nominal Spotsize (nm)',
    'Spotsize (nm)'
])
calibrations = calibrations.append([magnifications, camera_lengths, step_sizes1, step_sizes2, step_sizes3, step_sizes4, step_sizes5, precession_angles1, precession_angles2, precession_angles3, precession_angles4, precession_angles5, probe_sizes1, probe_sizes2, probe_sizes3, probe_sizes4, probe_sizes5])
calibrations

,Mode,Alpha,Nominal Mag,Magnification,Scale (nm/px),Image Rotation (deg),Nominal Camera Length (cm),Camera Length (cm),Scale (1/Å/px),Nominal Step Size X (nm),...,Precession Angle (deg),Precession Eccentricity,Nominal Spotsize (nm),Spotsize (nm),Mag Mode,Image,Date,Operator,Specimen,HT
0,NaN,NaN,8000.0,14918.0,3.69,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MAG,None,None,None,None,200000.0
1,NaN,NaN,10000.0,18483.0,2.98,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MAG,None,None,None,None,200000.0
2,NaN,NaN,12000.0,21624.0,2.54,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MAG,None,None,None,None,200000.0
3,NaN,NaN,15000.0,26827.0,2.05,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MAG,None,None,None,None,200000.0
4,NaN,NaN,20000.0,35247.0,1.56,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MAG,None,None,None,None,200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NBD,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.7,NaN,NaN,None,None,None,None,200000.0
2,NBD,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,None,None,None,None,200000.0
3,NBD,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.2,NaN,NaN,None,None,None,None,200000.0
4,NBD,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.4,NaN,NaN,None,None,None,None,200000.0


In [74]:
calibrations.query('`Nominal Step Size X (nm)` == {sx} | `Nominal Step Size Y (nm)` == {sy}'.format(sx=0.1, sy=0.2)).reset_index(drop=True)

,Mode,Alpha,Nominal Mag,Magnification,Scale (nm/px),Image Rotation (deg),Nominal Camera Length (cm),Camera Length (cm),Scale (1/Å/px),Nominal Step Size X (nm),...,Scan Rotation (deg),Nominal Precession Angle (deg),Precession Angle (deg),Precession Eccentricity,Mag Mode,Image,Date,Operator,Specimen,HT
0,NBD,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
1,NBD,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
2,NBD,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
3,NBD,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
4,NBD,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
5,NBD,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
6,NBD,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
7,NBD,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
8,NBD,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0
9,NBD,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0


In [91]:
names = ('X', 'Y')
shape = (1, 2, 3, 4)
axis_names = {0: {'name': ''},
              1: {'name': ''},
              2: {'name': ''},
              3: {'name': ''},
             }
[axis_names.update({len(shape) - 2 + ax: {'name':  name}}) for ax, name in enumerate(names)]

[None, None]

In [92]:
axis_names

{0: {'name': ''}, 1: {'name': ''}, 2: {'name': 'X'}, 3: {'name': 'Y'}}

In [83]:
a = [1, None]
[v for v in a if v is not None]

[1]

In [88]:
tuple([])

()

In [84]:
a+b

TypeError: can only concatenate list (not "tuple") to list

In [75]:
a = (1, 2)
b = (3, 4)

In [80]:
A = list(a)
B = list(b)
[A.append(e) for e in B]

[None, None]

In [81]:
A

[1, 2, 3, 4]

In [69]:
calibrations[calibrations.isin({'Nominal Step Size X (nm)': [0.1], 'Nominal Step Size Y (nm)': [0.2]})]

,Mode,Alpha,Nominal Mag,Magnification,Scale (nm/px),Image Rotation (deg),Nominal Camera Length (cm),Camera Length (cm),Scale (1/Å/px),Nominal Step Size X (nm),...,Scan Rotation (deg),Nominal Precession Angle (deg),Precession Angle (deg),Precession Eccentricity,Mag Mode,Image,Date,Operator,Specimen,HT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df = calibrations[calibrations['Nominal Step Size Y (nm)'] == 0.1]
df = df[df['Alpha'] == 5]

In [51]:
df['Step Size Y (nm)']

0    0.099
Name: Step Size Y (nm), dtype: float64

In [9]:
series = calibrations[calibrations['Nominal Mag']==8E3].copy()

In [10]:
series

,Mode,Alpha,Nominal Mag,Magnification,Scale X (nm/px),Scale Y (nm/px),Image Rotation (deg),Nominal Camera Length (cm),Camera Length (cm),Scale X (1/nm/px),...,Step Size Y (nm),Scan Rotation (deg),Nominal Precession Angle (deg),Precession Angle X (deg),Precession Angle Y (deg),Image,Date,Operator,Specimen,HT
0,Mag1,NaN,8000.0,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0


In [57]:
a = {'name': 'test'}

In [58]:
a['name']

'test'

In [59]:
a = {0:{'name':'x', 'scale':1}, 1:{'name': 'y'}}

In [63]:
n = a[1].get('scale', None)

In [65]:
type(n)

NoneType

In [14]:
val=series['HT']

In [16]:
float(val)

200000.0

In [55]:
calibrations.to_csv('Calibrations.csv')

In [ ]:
camera_length_series = pd.Series

In [ ]:
calibrations

In [68]:
calibrations['Nominal Step Size X (nm)']==0.5

0    False
1    False
2    False
3    False
4    False
     ...  
4     True
5    False
6    False
7    False
8    False
Name: Nominal Step Size X (nm), Length: 72, dtype: bool

In [84]:
test = {'a': 1, 'b':2}

In [87]:
calibrations['mode']

KeyError: 'mode'

In [88]:
calibrations.copy()

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)
0,Mag1,NaN,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
1,Mag1,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
2,Mag1,NaN,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
3,Mag1,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,Mag1,NaN,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,NBD,5.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
5,NBD,5.0,NaN,NaN,NaN,NaN,0.6,0.6,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
6,NBD,5.0,NaN,NaN,NaN,NaN,0.7,0.7,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
7,NBD,5.0,NaN,NaN,NaN,NaN,0.8,0.8,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN


In [85]:
test.popitem()

('b', 2)

In [78]:
test.pop('a')

1

In [83]:
list(test.keys())[0]

'b'

In [79]:
test

{'b': 2}

In [75]:
calibrations.loc[(calibrations['Nominal Step Size X (nm)']==0.5) & (calibrations['Mode'].isin(['NBD', 'TEM']))]

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)
4,NBD,1.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,NBD,2.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,NBD,3.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,NBD,4.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,NBD,5.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN


In [12]:
def look_up(table, parameter, value, filter_table={}):
    result = table.loc[table[parameter] == value]
    if len(filter_table) > 0:
        try:
            sub_result = result
            for filter_parameter in filter_table:
                sub_result = sub_result.loc[sub_result[filter_parameter] == filter_table[filter_parameter]]
        except Exception as e:
            print('Could not subfilter calibration table:\n{e}'.format(e=e))
        else:
            result = sub_result
    return result

In [32]:
today = dt.datetime.today()
yesterday = dt.datetime(2020,9,16)
someday = dt.datetime(2020,8,31)
difference1 = (today-yesterday)
difference2 = (today-someday)

In [34]:
min([today, yesterday])

datetime.datetime(2020, 9, 16, 0, 0)

datetime.datetime(2020, 9, 16, 0, 0)

In [21]:
today-dt.datetime('2020.09.16')

TypeError: an integer is required (got type str)

In [58]:
c = look_up(calibrations, 'Nominal Step Size X (nm)', 0.1)
c

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)
0,NBD,1.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,2.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,3.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,4.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,5.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN


In [57]:
calibrations

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)
0,Mag1,NaN,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
1,Mag1,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
2,Mag1,NaN,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
3,Mag1,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,Mag1,NaN,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,NBD,5.0,NaN,NaN,NaN,NaN,0.5,0.5,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
5,NBD,5.0,NaN,NaN,NaN,NaN,0.6,0.6,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
6,NBD,5.0,NaN,NaN,NaN,NaN,0.7,0.7,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
7,NBD,5.0,NaN,NaN,NaN,NaN,0.8,0.8,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN


In [56]:
c

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)
0,NBD,1.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,2.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,3.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,4.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
0,NBD,5.0,NaN,NaN,NaN,NaN,0.1,0.1,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN


In [46]:
c.loc[c['Date']=='None']

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)


In [40]:
c.at[c['Date'].index]

ValueError: Invalid call for scalar access (getting)!

In [ ]:
c.loc[c['Date']=min(c['Date'])]

In [11]:
calibrations.loc[calibrations['Mode']=='Mag1']

,Mode,Alpha,Nominal Mag,Magnification,Nominal Camera Length (cm),Camera Length (cm),Nominal Step Size X (nm),Nominal Step Size Y (nm),Scale X (nm/px),Scale Y (nm/px),Image rotation (deg),Image,Date,Operator,Specimen,HT,Scale X (1/nm/px),Scale Y (1/nm/px)
0,Mag1,NaN,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
1,Mag1,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
2,Mag1,NaN,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
3,Mag1,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
4,Mag1,NaN,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
5,Mag1,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
6,Mag1,NaN,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
7,Mag1,NaN,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
8,Mag1,NaN,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN
9,Mag1,NaN,70000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,200000.0,NaN,NaN


## TEM Magnification Calibration
To add (or correct) a TEM magnification calibration, acquire a useable TEM image and load it. Calibrate the image using e.g. pyxem, and get the scale (and the magnification) from the `axes_manager`. Then, either find the old calibration in the table and update it, or add it as a new calibration. If you add a new calibration to a table that already contains a calibration for the same magnification, you should indicate the date of your calibration. 

Load image

In [105]:
path = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Calibrations\2100F\TEM_Mag1_1M.dm4')
image = hs.load(str(path))

Do diffractogram analysis

In [127]:
fft = image.fft(shift=True)
power_spectrum = hs.signals.Signal2D(10 * np.log10(np.abs(fft.data)**2))
power_spectrum.plot()
circle = hs.roi.CircleROI(int(power_spectrum.axes_manager[0].size/2),
                          int(power_spectrum.axes_manager[1].size/2),
                          int(power_spectrum.axes_manager[0].size/4))
circle.add_widget(power_spectrum)

Calculate the scale

In [136]:
r = circle.r #distance between symmetric frequencies in diffractogram
a = 0.405 #lattice parameter in nm
h, k, l = 2, 0, 0 #Miller indices of diffractogram frequency planes - should correspond to the frequency the ring intersects, we will only use the radius
miller = np.array([h, k, l])
d_hkl = sum(np.sqrt((miller/a)**2))
scale = d_hkl/r #Scale of image in nm/px
print('Radius of ({h}{k}{l}) ring in diffractogram: {r:.2f} px\nActual distance: {d:.2f} nm\nScale in image: {scale:.2e} nm/px'.format(h=h,k=k,l=l,r=r,d=d_hkl,scale=scale))

Radius of (200) ring in diffractogram: 207.50 px
Actual distance: 4.94 nm
Scale in image: 2.38e-02 nm/px


Apply the calibration

datetime.date(2020, 9, 8)

In [218]:
image.axes_manager[0].scale = scale
image.axes_manager[0].units = 'nm'
image.axes_manager[1].scale = scale
image.axes_manager[1].units = 'nm'
#calibrations.at[calibrations[calibrations['Nominal Mag'] == 50000].index.values[0], 'Image'] = image
calibrations.at[calibrations[calibrations['Nominal Mag'] == 50000].index.values[0], 'Scale X (nm/px)'] = image.axes_manager[0].scale
calibrations.at[calibrations[calibrations['Nominal Mag'] == 50000].index.values[0], 'Scale Y (nm/px)'] = image.axes_manager[1].scale
calibrations.at[calibrations[calibrations['Nominal Mag'] == 50000].index.values[0], 'Date'] = dt.date.today()

In [220]:
calibrations[7]

KeyError: 7

In [207]:
ind = calibrations[calibrations['Nominal Mag'] == 50000].index.values

In [209]:
ind.values[0]

7

In [211]:
calibrations.at[1, 'Scale X (nm/px)']

1   NaN
1   NaN
1   NaN
1   NaN
1   NaN
1   NaN
1   NaN
Name: Scale X (nm/px), dtype: float64

7   NaN
7   NaN
7   NaN
7   NaN
7   NaN
7   NaN
7   NaN
Name: Scale X (nm/px), dtype: float64

In [184]:
calibrations.at[0, 'Scale X (nm/px)'] = -1

In [174]:
calibrations.iat[calibrations[calibrations['Nominal Mag'] == 50000].index, 0]

ValueError: iAt based indexing can only have integer indexers

In [138]:
image.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acquisition_mode = TEM
│       ├── beam_current = 0.0
│       ├── beam_energy = 200.0
│       ├── magnification = 500000.0
│       └── microscope = JEOL COM
├── General
│   ├── authors = E. Christiansen
│   ├── date = 2018-08-23
│   ├── original_filename = TEM_Mag1_1M.dm4
│   ├── time = 18:03:58
│   └── title = 2018_08_23_AA6060T6WQ_0-6_0001
├── Sample
│   └── description = AA6060T6WQ_5-A4
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 1.0
    │       └── gain_offset = 0.0
    ├── binned = False
    └── signal_type = 